# Super Resolution with PaddleGAN and DeGirum PySDK

This notebook demonstrates the RealSR (real-world super-resolution) model from [PaddlePaddle/PaddleGAN](https://github.com/PaddlePaddle/PaddleGAN) using OpenVINO through DeGirum PySDK.

For more information about the various PaddleGAN superresolution models, refer to the [PaddleGAN documentation](https://github.com/PaddlePaddle/PaddleGAN/blob/develop/docs/en_US/tutorials/single_image_super_resolution.md). For more information about RealSR, see the [research paper](https://openaccess.thecvf.com/content_CVPRW_2020/papers/w31/Ji_Real-World_Super-Resolution_via_Kernel_Estimation_and_Noise_Injection_CVPRW_2020_paper.pdf) from CVPR 2020.

This notebook works best with small images (up to 600x600 resolution).

In [1]:
# make sure PySDK and degirum-tools is installed
!pip show degirum || pip install -q degirum
!pip show degirum-tools || pip install -q degirum-tools

Name: degirum
Version: 0.12.0
Summary: DeGirum AI Inference Software Package
Home-page: https://github.com/degirum
Author: DeGirum Corp.
Author-email: support@degirum.com
License: 
Location: /home/mehrdad/.local/lib/python3.10/site-packages
Requires: msgpack, msgpack-numpy, numpy, opencv-python, pillow, python-socketio, pyyaml, requests, types-PyYAML, types-requests, websocket-client
Required-by: degirum-tools
Name: degirum-tools
Version: 0.5.2
Summary: Tools for PySDK
Home-page: UNKNOWN
Author: DeGirum
Author-email: 
License: UNKNOWN
Location: /usr/local/lib/python3.10/dist-packages
Requires: degirum, ffmpegcv, ipython, numpy, opencv-python, pafy, pillow, psutil, pycocotools, pyyaml, requests, scipy, youtube-dl
Required-by: 


In [2]:
import cv2
import degirum as dg
import degirum_tools as dgtools
import numpy as np
from PIL import Image

from superres_postprocessor import SuperResolutionResults, preprocess

hw_location = dg.CLOUD
zoo_name = 'https://cs.degirum.com/degirum/super_resolution'
model_name = 'paddlegan_realsr--600x600_float_openvino_cpu_1'

## Run inference on an image

In [3]:

zoo = dg.connect(hw_location, zoo_name, dgtools.get_token())
model = zoo.load_model(model_name, image_backend="pil", input_image_format="RAW")

# Set the timeout longer due to model inference time.
model.inference_timeout_s = 180

# Set up the custom postprocessor.
SuperResolutionResults.resize_factor = 4
model.custom_postprocessor = SuperResolutionResults

# Preprocess the image.
input = cv2.imread('data/coco_tulips.jpg')
processed_input = preprocess(input, model)

results = model(processed_input)

results.image_overlay

[ WARN:0@2.898] global loadsave.cpp:248 findDecoder imread_('data/coco_tulips.jpg'): can't open/read file: check file path/integrity


Exception: Image type not supported.

## Compare super resolution results to bicubic interpolation

In [ ]:
h, w, _ = input.shape
bicubic_image = cv2.resize(input, (w * 4, h * 4), interpolation=cv2.INTER_CUBIC)

# Stack bicubic image above super resolution image.
compound_image = np.zeros((h * 4 * 2, w * 4, 3), dtype=np.uint8)

compound_image[:h * 4, :w * 4] = cv2.cvtColor(bicubic_image, cv2.COLOR_BGR2RGB)
compound_image[h * 4:, :w * 4] = np.array(results.image_overlay)

Image.fromarray(compound_image)